In [1]:
import pandas as pd

In [2]:
pkl=r'C:\Users\andrew\Desktop\mosconi\new_inc_2015_fixed_again'
df= pd.read_pickle(pkl)

In [3]:
df['Europe_won']=df.European_score>df.American_score

In [4]:
df.describe()

,Format,European_player,European_score,American_score,American_player,Europe_won
count,335,335,335,335,335,335
unique,5,102,8,8,101,2
top,Singles,Mika Immonen,5,5,Johnny Archer,True
freq,183,27,126,122,28,169


In [5]:
df.Format.unique()

array(['Doubles', 'Singles', 'Teams', 'Trebles', 'Triples'], dtype=object)

In [6]:
trips = df.Format =='Triples' 
df[trips]

,Format,European_player,European_score,American_score,American_player,Europe_won
16,Triples,Imran Majid Thomas Engert Nick van den Berg,5,6,Corey Deuel Earl Strickland John Schmidt,False
17,Triples,Ralf Souquet Mika Immonen David Alcaide,1,6,Johnny Archer Rodney Morris Mike Davis,False


Note: Found it unusual that there are five types of format. Turns out that besides the normal match types (doubles, singles, and teams) there are also four triples matches: two named 'Triples' and two named 'Trebles'

Find a way to loop thru all players and make boolean queries to return their matches played

In [7]:
#Create lists of players on each team
with open('euro_players_scrape.txt', 'r') as rf:
    euro_players=[]
    for line in rf:
        n=line.split('  ')
        euro_players.append(n[0])
with open('am_players_scrape.txt', 'r') as rf:
    am_players=[]
    for line in rf:
        n=line.split('  ')
        am_players.append(n[0])

In [8]:
#remove non-playing captains and fix oscar's name
am_players.remove('Ã“scar DomÃ\xadnguez')
am_players.append('Óscar Domínguez')
am_players.remove('Buddy Hall')
# am_players.remove('Don Wardell')
euro_players.remove('Johan Ruijsink')

In [9]:
#Create shorthands for boolean indexing the dataframe
singles = df['Format']=='Singles'
doubles = df['Format']=='Doubles'
teams = df['Format']=='Teams'
europe_won = df['Europe_won'] == True
america_won = df['Europe_won'] == False

In [10]:
#create dictionary to boolean index df by player
#*note: str.contains allows matches for when there are other characters as well, such as in doubles 
d={}
for player in am_players:
    d[player] = df['American_player'].str.contains(player)
for player in euro_players:
    d[player] = df['European_player'].str.contains(player)

In [11]:
players = am_players + euro_players

In [12]:
c=0
f=0
nodata=[]
hasdata=[]
for player in players:
    _=df[d[player]]
    if _.empty:
        f+=1
        nodata.append(player)
    else:
        c+=1
        hasdata.append(player)
print(c,f)

49 28


The following players have no matches from 1998-2015 (they played either between 1994-1998 or in 2016):

In [13]:
nodata

['Mark Wilson',
 'Lou Butera',
 'Mike Gulyassy',
 'Robert Hunter',
 'Mike Massey',
 'Dallas West',
 'C.J. Wiley',
 'John DiToro',
 'Paul Gerni',
 'Roger Griffis',
 'Danny Harriman',
 'Allen Hopkins',
 'Bobby Hunter',
 'Jeanette Lee',
 'Vivian Villarreal',
 'Don Wardell ',
 "Ronnie O'Sullivan",
 'Tom Storm',
 'Jimmy White',
 'Tommy Donlon',
 'Vincent Facquet',
 'Allison Fisher',
 'Alex Higgins',
 'Lee Kendall',
 'Andy Richardson',
 'Jayson Shaw',
 'Franziska Stark',
 'Lee Tucker']

In [14]:
def singleswinloss(player,country):
    if 'a' in country:
        win=df[america_won&singles&d[player]].shape[0]
        loss=df[d[player]&~america_won&singles].shape[0]
    else:
        loss=df[d[player]&america_won&singles].shape[0]
        win=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(win,loss)
    return(swl)
def doubleswinloss(player,country):
    if 'a' in country:
        win=df[d[player]&america_won&doubles].shape[0]
        loss=df[d[player]&~america_won&doubles].shape[0]
    else:
        loss=df[d[player]&america_won&doubles].shape[0]
        win=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(win,loss)
    return(dwl)
def combinedwinloss(player,country):
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        swin=df[d[player]&america_won&singles].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0]
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]
    win=dwin+swin
    loss=dloss+sloss
    cwl='{}-{}'.format(win,loss)
    return(cwl)

In [15]:
def winloss(player, country):
    
    #singles
    if 'a' in country:
        win=df[america_won&singles&d[player]].shape[0]
        loss=df[d[player]&~america_won&singles].shape[0]
    else:
        loss=df[d[player]&america_won&singles].shape[0]
        win=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(win,loss)
    spct=win*100/(win+loss)

    #doubles
    if 'a' in country:
        win=df[d[player]&america_won&doubles].shape[0]
        loss=df[d[player]&~america_won&doubles].shape[0]
    else:
        loss=df[d[player]&america_won&doubles].shape[0]
        win=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(win,loss)
    dpct=win*100/(win+loss)

    #combined
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
        swin=df[d[player]&america_won&singles].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0]
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]
    win=dwin+swin
    loss=dloss+sloss
    cwl='{}-{}'.format(win,loss)
    cpct=win*100/(win+loss)
    print("singles: {}   pct:{}   doubles: {}  pct:{}    combined: {}  pct:{}".format(swl,spct,dwl,dpct,cwl,cpct))
    return(swl,dwl,cwl)

In [16]:
winloss('Johnny Archer','am')

singles: 15-13   pct:53.57142857142857   doubles: 27-20  pct:57.4468085106383    combined: 42-33  pct:56.0


('15-13', '27-20', '42-33')

In [17]:
def winlosslist(player, country):
    
    #singles
    if 'a' in country:
        swin=df[america_won&singles&d[player]].shape[0]
        sloss=df[d[player]&~america_won&singles].shape[0]
    else:
        sloss=df[d[player]&america_won&singles].shape[0]
        swin=df[d[player]&~america_won&singles].shape[0]        
    swl='{}-{}'.format(swin,sloss)
    spct=swin*100/(swin+sloss)

    #doubles
    if 'a' in country:
        dwin=df[d[player]&america_won&doubles].shape[0]
        dloss=df[d[player]&~america_won&doubles].shape[0]
    else:
        dloss=df[d[player]&america_won&doubles].shape[0]
        dwin=df[d[player]&~america_won&doubles].shape[0] 
    dwl='{}-{}'.format(dwin,dloss)
    dpct=dwin*100/(dwin+dloss)

    #combined
    cwin=dwin+swin
    closs=dloss+sloss
    cwl='{}-{}'.format(cwin,closs)
    cpct=cwin*100/(cwin+closs)
    l=[player,country,swin,sloss,swl,round(spct,2),dwin,dloss,dwl,round(dpct,2),cwin,closs,cwl,round(cpct,2)]
    return(l)

In [18]:
winlosslist('Niels Feijen','eu')

['Niels Feijen',
 'eu',
 11,
 8,
 '11-8',
 57.89,
 22,
 9,
 '22-9',
 70.97,
 33,
 17,
 '33-17',
 66.0]

In [19]:
L=[]
for player in am_players:
    try:
        lst = winlosslist(player,'America')
        L.append(lst)
    except: #Zerodivision error:
        pass
for player in euro_players:
    try:
        lst = winlosslist(player,'Europe')
        L.append(lst)
    except: #Zerodivision error:
        pass

For player in players, append to l, create winlosslist, then turn into dataframe with stats of all players.

In [20]:
import pandas as pd
colz = 'Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Cwin,Closs,Cwl,Cpct'.split(',')
player_stats = pd.DataFrame(L, columns = colz)
player_stats.sort_values('Cpct',axis=0)

,Player,Team,Swin,Sloss,Swl,Spct,Dwin,Dloss,Dwl,Dpct,Cwin,Closs,Cwl,Cpct
18,John Schmidt,America,1,2,1-2,33.33,0,5,0-5,0.00,1,7,1-7,12.50
23,Brandon Shuff,America,0,3,0-3,0.00,1,2,1-2,33.33,1,5,1-5,16.67
43,Daryl Peach,Europe,0,2,0-2,0.00,1,2,1-2,33.33,1,4,1-4,20.00
47,Fabio Petroni,Europe,0,2,0-2,0.00,1,2,1-2,33.33,1,4,1-4,20.00
8,Mike Dechaine,America,1,4,1-4,20.00,2,6,2-6,25.00,3,10,3-10,23.08
45,Raj Hundal,Europe,1,0,1-0,100.00,0,3,0-3,0.00,1,3,1-3,25.00
38,Alex Lely,Europe,0,2,0-2,0.00,2,4,2-4,33.33,2,6,2-6,25.00
36,Mark Gray,Europe,0,1,0-1,0.00,2,4,2-4,33.33,2,5,2-5,28.57
35,Thomas Engert,Europe,1,5,1-5,16.67,3,5,3-5,37.50,4,10,4-10,28.57
11,Justin Bergman,America,1,4,1-4,20.00,2,2,2-2,50.00,3,6,3-6,33.33


## Mosconi Cup Trivia Questions
Most wins? (s,d,c)
Highest win %? (s,d,c)
Lowest win %? (s,d,c)


In [21]:
def playerinfo(player,country):
    '''get info about player stats in a list and output every match'''
    wl=winlosslist(player,country)
    frame=df[d[player]]
    print((winlosslist(player,country)))
    return(frame)

In [22]:
_=playerinfo('Niels Feijen','f')
_[_['Format']=='Singles']

['Niels Feijen', 'f', 11, 8, '11-8', 57.89, 22, 9, '22-9', 70.97, 33, 17, '33-17', 66.0]


,Format,European_player,European_score,American_score,American_player,Europe_won
10,Singles,Niels Feijen,2,5,Earl Strickland,False
7,Singles,Niels Feijen,1,5,Charlie Williams,False
16,Singles,Niels Feijen,5,2,Johnny Archer,True
3,Singles,Niels Feijen,3,5,Charlie Williams,False
4,Singles,Niels Feijen,1,6,Rodney Morris,False
13,Singles,Niels Feijen,6,4,Rodney Morris,True
11,Singles,Niels Feijen,3,5,Rodney Morris,False
11,Singles,Niels Feijen,4,6,Dennis Hatch,False
15,Singles,Niels Feijen,0,6,Dennis Hatch,False
2,Singles,Niels Feijen,6,0,Rodney Morris,True


In [23]:
print(playerinfo.__doc__)

get info about player stats in a list and output every match


In [24]:
stat_l = []
for player in am_players:
    try:
        unit = winlosslist(player,'America')
        stat_l.append(unit)
    except:
        pass
for player in euro_players:
    try:
        unit = winlosslist(player,'Europe')
        stat_l.append(unit)
    except:
        pass

In [25]:
stat_cols = ['player','team','s_wins','s_loss', 's_winloss', 's_pct','d_wins','d_loss', 'd_winloss', 'd_pct','c_wins','c_loss', 'c_winloss', 'c_pct']

In [26]:
#Most combined wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','c_winloss','c_pct','c_wins']].sort_values('c_wins',ascending=False).reset_index(drop=True)

,player,c_winloss,c_pct,c_wins
0,Johnny Archer,42-33,56.00,42
1,Ralf Souquet,38-25,60.32,38
2,Niels Feijen,33-17,66.00,33
3,Mika Immonen,32-30,51.61,32
4,Earl Strickland,31-21,59.62,31
5,Shane Van Boening,21-25,45.65,21
6,Rodney Morris,20-23,46.51,20
7,Corey Deuel,20-18,52.63,20
8,Jeremy Jones,17-16,51.52,17
9,Darren Appleton,17-17,50.00,17


In [27]:
#Best combined win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','c_winloss','c_pct']].sort_values('c_pct',ascending=False).head(20).reset_index(drop=True)

,player,c_winloss,c_pct
0,Albin Ouschan,4-1,80.00
1,Gabe Owen,4-1,80.00
2,Kim Davenport,8-2,80.00
3,Karl Boyes,14-4,77.78
4,Justin Hall,3-1,75.00
5,Niels Feijen,33-17,66.00
6,Ralf Souquet,38-25,60.32
7,Chris Melling,6-4,60.00
8,Nick Ekonomopoulos,3-2,60.00
9,Shannon Daulton,3-2,60.00


In [28]:
#Most singles wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','s_winloss','s_wins']].sort_values('s_wins',ascending=False).head(11).reset_index(drop=True)

,player,s_winloss,s_wins
0,Johnny Archer,15-13,15
1,Ralf Souquet,15-10,15
2,Mika Immonen,14-13,14
3,Earl Strickland,11-9,11
4,Niels Feijen,11-8,11
5,Shane Van Boening,10-14,10
6,Rodney Morris,9-8,9
7,Darren Appleton,9-4,9
8,Jeremy Jones,7-7,7
9,Corey Deuel,7-6,7


In [29]:
playerinfo('Justin Hall','am')

['Justin Hall', 'am', 1, 0, '1-0', 100.0, 2, 1, '2-1', 66.67, 3, 1, '3-1', 75.0]


,Format,European_player,European_score,American_score,American_player,Europe_won
1,Doubles,Darren Appleton & Karl Boyes,3,5,Corey Deuel & Justin Hall,False
5,Doubles,Niels Feijen & Mark Gray,2,5,Justin Hall & Justin Bergman,False
10,Doubles,Nikos Ekonomopoulos & Karl Boyes,5,3,John Schmidt & Justin Hall,True
11,Singles,Nikos Ekonomopoulos,3,5,Justin Hall,False


In [30]:
playerinfo('Justin Bergman','am')

['Justin Bergman', 'am', 1, 4, '1-4', 20.0, 2, 2, '2-2', 50.0, 3, 6, '3-6', 33.33]


,Format,European_player,European_score,American_score,American_player,Europe_won
3,Doubles,Niels Feijen & Nikos Ekonomopoulos,5,1,Shane Van Boening & Justin Bergman,True
5,Doubles,Niels Feijen & Mark Gray,2,5,Justin Hall & Justin Bergman,False
8,Singles,Niels Feijen,5,3,Justin Bergman,True
13,Singles,Niels Feijen,5,2,Justin Bergman,True
2,Singles,Niels Feijen,5,2,Justin Bergman,True
5,Doubles,Niels Feijen Nick van den Berg,3,5,Justin Bergman Skyler Woodward,False
10,Doubles,Niels Feijen Albin Ouschan,5,1,Justin Bergman Corey Deuel,True
14,Singles,Darren Appleton,2,5,Justin Bergman,False
15,Singles,Niels Feijen,5,4,Justin Bergman,True


In [31]:
stats_df.head()

,player,team,s_wins,s_loss,s_winloss,s_pct,d_wins,d_loss,d_winloss,d_pct,c_wins,c_loss,c_winloss,c_pct
0,Johnny Archer,America,15,13,15-13,53.57,27,20,27-20,57.45,42,33,42-33,56.00
1,Earl Strickland,America,11,9,11-9,55.00,20,12,20-12,62.50,31,21,31-21,59.62
2,Rodney Morris,America,9,8,9-8,52.94,11,15,11-15,42.31,20,23,20-23,46.51
3,Shane Van Boening,America,10,14,10-14,41.67,11,11,11-11,50.00,21,25,21-25,45.65
4,Corey Deuel,America,7,6,7-6,53.85,13,12,13-12,52.00,20,18,20-18,52.63


In [32]:
#Best singles win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','s_wins','s_winloss','s_pct']].sort_values(['s_pct','s_wins'],ascending=[False,False]).head(28).reset_index(drop=True)
# df.sort_values(['a', 'b'], ascending=[True, False])

,player,s_wins,s_winloss,s_pct
0,Chris Melling,5,5-0,100.00
1,Kim Davenport,4,4-0,100.00
2,Reed Pierce,2,2-0,100.00
3,Gabe Owen,2,2-0,100.00
4,Albin Ouschan,2,2-0,100.00
5,Justin Hall,1,1-0,100.00
6,Raj Hundal,1,1-0,100.00
7,Charlie Williams,6,6-2,75.00
8,Darren Appleton,9,9-4,69.23
9,Karl Boyes,4,4-2,66.67


In [33]:
#Most doubles wins
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','d_winloss','d_wins']].sort_values('d_wins',ascending=False).reset_index(drop=True).head(10)

,player,d_winloss,d_wins
0,Johnny Archer,27-20,27
1,Ralf Souquet,23-15,23
2,Niels Feijen,22-9,22
3,Earl Strickland,20-12,20
4,Mika Immonen,18-17,18
5,Corey Deuel,13-12,13
6,Rodney Morris,11-15,11
7,Shane Van Boening,11-11,11
8,Jeremy Jones,10-9,10
9,Karl Boyes,10-2,10


In [34]:
#Best doubles win percentage
stats_df = pd.DataFrame(data=stat_l, columns = stat_cols)
stats_df[['player','d_winloss','d_pct']].sort_values('d_pct',ascending=False).head(20).reset_index(drop=True)

,player,d_winloss,d_pct
0,Karl Boyes,10-2,83.33
1,Niels Feijen,22-9,70.97
2,Óscar Domínguez,2-1,66.67
3,Shannon Daulton,2-1,66.67
4,Nick Ekonomopoulos,2-1,66.67
5,Albin Ouschan,2-1,66.67
6,Gabe Owen,2-1,66.67
7,Justin Hall,2-1,66.67
8,Kim Davenport,4-2,66.67
9,Skyler Woodward,2-1,66.67


#Info to verify from the AZ billiards forums:
Team USA
• Shane Van Boening (9 years -- 2007-2015) ..... 10-14 singles, 11-11 doubles, 4-5 teams, 25-30 total (winning percentage of 45%)

• Rodney Morris (9 years -- 2003-2008, 2010, 2011, 2013) ..... 9-8 singles, 11-15 doubles, 2-0 trebles, 3-4 teams, 25-27 total (48%)

• Mike Dechaine (3 years -- 2011, 2012, 2015) ..... 1-4 singles, 2-6 doubles, 2-1 teams, 5-11 total (31%)

• Justin Bergman (2 years -- 2014-2015) ..... 1-4 singles, 2-2 doubles, 1-1 teams, 4-7 total (36%)

• Skyler Woodward (1 year -- 2015) ..... 0-2 singles, 2-1 doubles, 1-0 teams, 3-3 total (50%)

• Team USA Total ..... 21-32 singles, 28-35 doubles, 2-0 trebles, 11-11 teams, 62-78 total (44%)

Team Europe
• Niels Feijen (11 years -- 2001, 2004-2005, 2007-2009, 2011-2015) ..... 11-8 singles, 22-9 doubles, 4-4 teams, 37-21 total (64%)

• Darren Appleton (7 years -- 2009-2015) ..... 9-4 singles, 8-13 doubles, 4-3 teams, 21-20 total (51%)

• Mark Gray (2 years -- 2008, 2014) ..... 0-1 singles, 2-4 doubles, 2-0 teams, 4-5 total (44%)

• Albin Ouschan (1 year -- 2015) ..... 2-0 singles, 2-1 doubles, 0-1 teams, 4-2 total (67%)

• Jayson Shaw (0 years -- rookie)

• Team Europe Total ..... 22-13 singles, 34-27 doubles, 10-8 teams, 66-48 total (58%)

In [35]:
usa = ['Shane Van Boening', 'Rodney Morris', 'Mike Dechaine', 'Justin Bergman', 'Skyler Woodward']
for player in usa:
    print(winlosslist(player, 'am'))

['Shane Van Boening', 'am', 10, 14, '10-14', 41.67, 11, 11, '11-11', 50.0, 21, 25, '21-25', 45.65]
['Rodney Morris', 'am', 9, 8, '9-8', 52.94, 11, 15, '11-15', 42.31, 20, 23, '20-23', 46.51]
['Mike Dechaine', 'am', 1, 4, '1-4', 20.0, 2, 6, '2-6', 25.0, 3, 10, '3-10', 23.08]
['Justin Bergman', 'am', 1, 4, '1-4', 20.0, 2, 2, '2-2', 50.0, 3, 6, '3-6', 33.33]
['Skyler Woodward', 'am', 0, 2, '0-2', 0.0, 2, 1, '2-1', 66.67, 2, 3, '2-3', 40.0]


In [36]:
eu = ['Darren Appleton', 'Mark Gray', 'Albin Ouschan', 'Niels Feijen']
for player in eu:
    print(winlosslist(player, 'u'))

['Darren Appleton', 'u', 9, 4, '9-4', 69.23, 8, 13, '8-13', 38.1, 17, 17, '17-17', 50.0]
['Mark Gray', 'u', 0, 1, '0-1', 0.0, 2, 4, '2-4', 33.33, 2, 5, '2-5', 28.57]
['Albin Ouschan', 'u', 2, 0, '2-0', 100.0, 2, 1, '2-1', 66.67, 4, 1, '4-1', 80.0]
['Niels Feijen', 'u', 11, 8, '11-8', 57.89, 22, 9, '22-9', 70.97, 33, 17, '33-17', 66.0]


#More info to verify:
• Johnny Archer (17 years) ..... 18-13 singles, 26-23 doubles, 0-2 trebles, 5-4 teams, 49-42 total (winning percentage of 54%)
• Earl Strickland (14 years) ..... 14-12 singles, 24-12 doubles, 1-1 trebles, 3-2 teams, 42-27 total (61%)
• Corey Deuel (9 years) ..... 7-6 singles, 13-12 doubles, 2-0 trebles, 5-2 teams, 27-20 total (57%)
And more:
Dennis Hatch (4 years -- 2009, 2010, 2012, 2013) ..... 4-3 singles, 5-7 doubles, 2-2 teams, 11-12 total (winning percentage 48%)

In [37]:
pl=['Johnny Archer','Earl Strickland','Corey Deuel','Dennis Hatch']
for player in pl:
    print(winlosslist(player,'am'))

['Johnny Archer', 'am', 15, 13, '15-13', 53.57, 27, 20, '27-20', 57.45, 42, 33, '42-33', 56.0]
['Earl Strickland', 'am', 11, 9, '11-9', 55.0, 20, 12, '20-12', 62.5, 31, 21, '31-21', 59.62]
['Corey Deuel', 'am', 7, 6, '7-6', 53.85, 13, 12, '13-12', 52.0, 20, 18, '20-18', 52.63]
['Dennis Hatch', 'am', 4, 3, '4-3', 57.14, 5, 7, '5-7', 41.67, 9, 10, '9-10', 47.37]


Johnny's info is wrong: he has two singles wins in 1997, so still missing one singles win. Doubles wins says 27-20 but apparently should be 26-23. He was 0-3 doubles in 97. So total would be 27-23. I have too many wins(by one)! 
So I recorded one too many singles win and one too few doubles win...
Could it be that a singles win was accidentally marked as a doubles win??

For Earl, I am missing 3 singles wins, 3 singles losses, and 4 doubles wins. He played in 1996 and 1997. In 96 he was 1-3 singles, 1-0 doubles. In 97, he was 2-0 on singles and 3-0 doubles. So overall, he is missing 3-3 and 4-0. Thus his data adds up with the missing years. So Earl is good.

Deuel is correct. Hatch is correct.

Visually inspected all years and 2015 is the only year that is backwards.

In [38]:
playerinfo('Skyler Woodward','am')

['Skyler Woodward', 'am', 0, 2, '0-2', 0.0, 2, 1, '2-1', 66.67, 2, 3, '2-3', 40.0]


,Format,European_player,European_score,American_score,American_player,Europe_won
3,Doubles,Darren Appleton Albin Ouschan,1,5,Shane Van Boening Skyler Woodward,False
5,Doubles,Niels Feijen Nick van den Berg,3,5,Justin Bergman Skyler Woodward,False
8,Singles,Karl Boyes,5,3,Skyler Woodward,True
12,Doubles,Karl Boyes Albin Ouschan,5,2,Mike Dechaine Skyler Woodward,True
17,Singles,Albin Ouschan,5,4,Skyler Woodward,True


Data including 2016

Team USA
• Shane Van Boening (10 years -- 2007-2016) ..... 10-15 singles, 11-14 doubles, 4-6 teams, 25-35 total (winning percentage of 42%)

• Rodney Morris (10 years -- 2003-2008, 2010-2011, 2013, 2016) ..... 10-8 singles, 12-17 doubles, 2-0 trebles, 3-5 teams, 27-30 total (47%)

• Mike Dechaine (4 years -- 2011-2012, 2015-2016) ..... 1-6 singles, 2-8 doubles, 2-2 teams, 5-16 total (24%)

• Justin Bergman (3 years -- 2014-2016) ..... 1-5 singles, 4-3 doubles, 1-2 teams, 6-10 total (38%)

• Skyler Woodward (2 years -- 2015-2016) ..... 0-3 singles, 3-3 doubles, 1-1 teams, 4-7 total (36%)

• Team USA Total ..... 22-37 singles, 32-45 doubles, 2-0 trebles, 11-16 teams, 67-98 total (41%)

Team Europe
• Niels Feijen (12 years -- 2001, 2004-2005, 2007-2009, 2011-2016) ..... 12-8 singles, 24-9 doubles, 5-4 teams, 41-21 total (66%)

• Darren Appleton (8 years -- 2009-2016) ..... 10-4 singles, 10-15 doubles, 5-3 teams, 25-22 total (53%)

• Mark Gray (3 years -- 2008, 2014, 2016) ..... 1-1 singles, 4-5 doubles, 3-0 teams, 8-6 total (57%)

• Albin Ouschan (2 years -- 2015-2016) ..... 3-0 singles, 5-1 doubles, 1-1 teams, 9-2 total (82%)

• Jayson Shaw (1 year -- 2016) ..... 1-1 singles, 1-1 doubles, 1-0 teams, 3-2 total (60%)

• Team Europe Total ..... 27-14 singles, 44-31 doubles, 15-8 teams, 86-53 total (62%)